# I. Monero Address Calculations

As mentioned in the overview Alice has previously used Monero, but her computer crashed and she wants to recover all of her Monero data. The journey for Alice starts with a mnemonic. Alice wrote down her mnemonic and wants to use Python to extract all of the information it contains. She is using the standard mnemonic format used in Monero, a 25 word phrase. Specifically Alice's 25 word mnemonic is: `joyous weekday gearbox slower dejected nutshell fuzzy empty wetsuit mighty bawled dummy tufts lawsuit moon bluntly whole cage punch else sayings prying hockey imbalance slower`.

_Note: The 25 word mnemonic is by far the most common format, and is the only format that will be explained._

A Monero mnemonic (25 words) can be decoded into five pieces of information. Each of those five pieces of information will be recovered using Python with explanatory code. The five pieces of information are:
* private spend key: this is just a really large integer
* private view key: this is just a really large integer
* public view key: this is a point on a curve in the 2-d plane with (x,y) coordinates, the coordinates are modified such that they look like a single large number
* public spend key: this is a point on a curve in the 2-d plane with (x,y) coordinates, the coordinates are modified such that they look like a single large number
* Monero address: the public view key and public spend key joined together in a special format

There are two primary goals for this section:
1. display in easy to comprehend code all of the steps to convert a mnemonic to public/private view/spend keys and Monero address
2. similarly, display how to convert a Monero address back into public view/spend keys


This section relies heavliy on the following source:
* [Zero to Monero: Second Edition; Chapter 2 and 4](https://www.getmonero.org/library/Zero-to-Monero-2-0-0.pdf)


**It is highly recommended that the reader read chapter 2 and chapter 4 of _Zero to Monero: Second Edition_ prior to reading the following sections.**

The sections below cover:
1. creating the mnemonic checksum
2. calculating the private spend key
3. calculating the private view key
4. calculating the public spend key
5. calculating the public view key
6. converting the public spend/view keys into a Monero address 
7. recovering public spend/view keys from a Monero address
8. identifying public key curve points (optional) 

### Summary (Introduction)
**What we start with:**
* mnemonic (25 words): `joyous weekday gearbox slower dejected nutshell fuzzy empty wetsuit mighty bawled dummy tufts lawsuit moon bluntly whole cage punch else sayings prying hockey imbalance slower`

**What we end up with:**
* private spend key
* private view key
* public view key
* public spend key
* Monero address

## 1. Creating the mnemonic checksum

A mnemonic is 25 words long but only the first 24 words are used to create public/private view/spend keys and the Monero address. The 25th word is selected during mnemonic creation in a very specific way from the 24 preceding words. This words acts as a check to help ensure the mnemonic is valid. Valid words included in a Monero mnemonic come from a predefined ordered word list that is contained within the Monero codebase. The english word list contains 1626 words and will be listed and used in later steps.

This section will use the first 24 words of Alice's mnemonic to calculate the 25th word and compare it against the 25th word present in the mnemonic she is using. Specifically: 
* the first 24 words are: `joyous weekday gearbox slower dejected nutshell fuzzy empty wetsuit mighty bawled dummy tufts lawsuit moon bluntly whole cage punch else sayings prying hockey imbalance`
* the 25th checksum word is: `slower` 
    * Notice that the checksum word is the fourth word in the list of 24 words.

*Note: There are wordlists for multiple languages. The mnemonic comes from the english word list which must be used for the upcoming steps.* 

In [ ]:
'''
Zero to Monero: Second Edition; Section 4.1
'''


'''
To calculate the checksum word this code performs the following:

  1. Extracts the first three letters from each word
  2. Joins all the letters together
  3. Calculates a crc32 hash of the letters
  4. Converts this to a number modulus 24
  5. Selects a word from the list of 24 using the value created in step 4
'''

# import required libraries (libraries specific to each section will be imported/re-imported for clarity)
from binascii import crc32


# define a function to calculate checksum
def calculate_checksum(mnemonic_24_words: str) -> str:
    
    # only the first three characters from each word are used in the calculation
    unique_prefix_length = 3
    
    # take first three letters from each word and concatenate into on big string
    first_chars_joined = "".join(word[: unique_prefix_length] for word in mnemonic_24_words)
    print(f"Inside function - first three characters from each of 24 words joined: {first_chars_joined}")
    
    # convert the string into a bytearray
    first_chars_joined_bytes = bytearray(first_chars_joined.encode("utf-8"))
    print(f"Inside function - above variable as a bytearray: {first_chars_joined_bytes}")

    # calculate a crc32 hash of the bytearray
    first_chars_joined_bytes_crc32 = crc32(first_chars_joined_bytes)
    print(f"Inside function - above variable hashed to 32 bytes: {first_chars_joined_bytes_crc32}")

    # calculate the modulus ("remainder") after dividing the above number by the length of the 24 word list (len = 24 obviously)
    checksum_position = first_chars_joined_bytes_crc32 % 24
    print(f"Inside function - modulus of above number and the 24 word list length: {checksum_position}")

    # use the checksum value to select a checksum word, REMEMBER lists are 0 indexed so number 1 equals second position in list    
    return mnemonic_24_words[checksum_position]


# store Alice's mnemonic as a string
alice_mnemonic = "joyous weekday gearbox slower dejected nutshell fuzzy empty wetsuit mighty bawled dummy tufts lawsuit moon bluntly whole cage punch else sayings prying hockey imbalance slower"

# convert the string to a list of words
alice_mnemonic = alice_mnemonic.split(" ")

# extract the checksum
alice_mnemonic_checksum = alice_mnemonic.pop(-1)

# print 24 word mnemonic and 25 word checksum
print(f"Alice's mnemonic is: {alice_mnemonic}")
print(f"Alice's calculated checksum word is: {calculate_checksum(alice_mnemonic)}")
print(f"Alice's mnemonic checksum is: {alice_mnemonic_checksum}")

## 2. Calculating the private spend key

The first 24 words in a mnemonic are selected from a wordlist. Each mnemonic word represents an index position of a word from the ordered wordlist and thus represent a number. For example the word `abducts` is the second word in the Monero english wordlist so the word `abducts` (using zero indexing) represents the number 1. The 24 words can be converted to 24 numbers and then those numbers can be transformed into one large number. The large number is the private spend key.  

More specifically the private spend key is 256 bits (32 bytes) of data. The 32 bytes of data can be broken into 8 groups of 4 bytes of data which represents, in total, 32 bytes. $(8\text{ blocks} \times4\text{ bytes per block} = 32\text{ bytes})$ Four bytes, or 32 bits, of data can represent $4294967296 = 2^{32}$ numbers. Thus the entire 32 bytes, or 256 bits, of data in the private spend key can be represented using 8 numbers that are between 0 and 4294967295 (a range of 4294967296 numbers).  

The 24 word mnemonic can be broken into 8 groups of 3 words. Each of the mnemonic words represents a number between 0 and 1625 (word list length 1626). Using the private spend key derivation method Monero uses, three numbers between 0 and 1625 can be used to represent up to 4298942375 numbers. $[1626 + (1626)\times1625 + (1626\times1626)\times1625 = 4298942375]$. Thus a group of 3 words can represent more numbers than is required to fill four bytes of data $(4298942375 \gt 4294967296)$.  

Three numbers between 0 and 1625 can be used to represent up to 4298942375 this way:
* First word: used to represent numbers been 0 and 1625
* Second word: used to calculate numbers between 0 and 2642250 $[1626\times0, 1626\times1625]$
* Third word: used to calculate numbers between 0 and 4296298500 $[(1626\times1626)\times0, (1626\times1626)\times1625]$

The three calculated numbers can be added to create a number between $0 + 0 + 0 = 0$ and $1625 + 2642250 + 4296298500 = 4298942375$. That range covers a litte more than 4 bytes of data.

In summary:
* 24 words = 8 groups of 3 words
* 3 words = 4 bytes of data
* 8 groups of words * 4 bytes of data per group = 32 bytes of data
* 32 bytes of data * 8 bits per byte = 256 bits of data (the private spend key is 256 bits of data)

**Note: The word list section follows directly below but is collapsed so it doesn't take up the entire screen. Run that section first to store the word list in a variable.**

In [ ]:
# create a list to hold the english word list of 1626 words
word_list = [
        "abbey",
        "abducts",
        "ability",
        "ablaze",
        "abnormal",
        "abort",
        "abrasive",
        "absorb",
        "abyss",
        "academy",
        "aces",
        "aching",
        "acidic",
        "acoustic",
        "acquire",
        "across",
        "actress",
        "acumen",
        "adapt",
        "addicted",
        "adept",
        "adhesive",
        "adjust",
        "adopt",
        "adrenalin",
        "adult",
        "adventure",
        "aerial",
        "afar",
        "affair",
        "afield",
        "afloat",
        "afoot",
        "afraid",
        "after",
        "against",
        "agenda",
        "aggravate",
        "agile",
        "aglow",
        "agnostic",
        "agony",
        "agreed",
        "ahead",
        "aided",
        "ailments",
        "aimless",
        "airport",
        "aisle",
        "ajar",
        "akin",
        "alarms",
        "album",
        "alchemy",
        "alerts",
        "algebra",
        "alkaline",
        "alley",
        "almost",
        "aloof",
        "alpine",
        "already",
        "also",
        "altitude",
        "alumni",
        "always",
        "amaze",
        "ambush",
        "amended",
        "amidst",
        "ammo",
        "amnesty",
        "among",
        "amply",
        "amused",
        "anchor",
        "android",
        "anecdote",
        "angled",
        "ankle",
        "annoyed",
        "answers",
        "antics",
        "anvil",
        "anxiety",
        "anybody",
        "apart",
        "apex",
        "aphid",
        "aplomb",
        "apology",
        "apply",
        "apricot",
        "aptitude",
        "aquarium",
        "arbitrary",
        "archer",
        "ardent",
        "arena",
        "argue",
        "arises",
        "army",
        "around",
        "arrow",
        "arsenic",
        "artistic",
        "ascend",
        "ashtray",
        "aside",
        "asked",
        "asleep",
        "aspire",
        "assorted",
        "asylum",
        "athlete",
        "atlas",
        "atom",
        "atrium",
        "attire",
        "auburn",
        "auctions",
        "audio",
        "august",
        "aunt",
        "austere",
        "autumn",
        "avatar",
        "avidly",
        "avoid",
        "awakened",
        "awesome",
        "awful",
        "awkward",
        "awning",
        "awoken",
        "axes",
        "axis",
        "axle",
        "aztec",
        "azure",
        "baby",
        "bacon",
        "badge",
        "baffles",
        "bagpipe",
        "bailed",
        "bakery",
        "balding",
        "bamboo",
        "banjo",
        "baptism",
        "basin",
        "batch",
        "bawled",
        "bays",
        "because",
        "beer",
        "befit",
        "begun",
        "behind",
        "being",
        "below",
        "bemused",
        "benches",
        "berries",
        "bested",
        "betting",
        "bevel",
        "beware",
        "beyond",
        "bias",
        "bicycle",
        "bids",
        "bifocals",
        "biggest",
        "bikini",
        "bimonthly",
        "binocular",
        "biology",
        "biplane",
        "birth",
        "biscuit",
        "bite",
        "biweekly",
        "blender",
        "blip",
        "bluntly",
        "boat",
        "bobsled",
        "bodies",
        "bogeys",
        "boil",
        "boldly",
        "bomb",
        "border",
        "boss",
        "both",
        "bounced",
        "bovine",
        "bowling",
        "boxes",
        "boyfriend",
        "broken",
        "brunt",
        "bubble",
        "buckets",
        "budget",
        "buffet",
        "bugs",
        "building",
        "bulb",
        "bumper",
        "bunch",
        "business",
        "butter",
        "buying",
        "buzzer",
        "bygones",
        "byline",
        "bypass",
        "cabin",
        "cactus",
        "cadets",
        "cafe",
        "cage",
        "cajun",
        "cake",
        "calamity",
        "camp",
        "candy",
        "casket",
        "catch",
        "cause",
        "cavernous",
        "cease",
        "cedar",
        "ceiling",
        "cell",
        "cement",
        "cent",
        "certain",
        "chlorine",
        "chrome",
        "cider",
        "cigar",
        "cinema",
        "circle",
        "cistern",
        "citadel",
        "civilian",
        "claim",
        "click",
        "clue",
        "coal",
        "cobra",
        "cocoa",
        "code",
        "coexist",
        "coffee",
        "cogs",
        "cohesive",
        "coils",
        "colony",
        "comb",
        "cool",
        "copy",
        "corrode",
        "costume",
        "cottage",
        "cousin",
        "cowl",
        "criminal",
        "cube",
        "cucumber",
        "cuddled",
        "cuffs",
        "cuisine",
        "cunning",
        "cupcake",
        "custom",
        "cycling",
        "cylinder",
        "cynical",
        "dabbing",
        "dads",
        "daft",
        "dagger",
        "daily",
        "damp",
        "dangerous",
        "dapper",
        "darted",
        "dash",
        "dating",
        "dauntless",
        "dawn",
        "daytime",
        "dazed",
        "debut",
        "decay",
        "dedicated",
        "deepest",
        "deftly",
        "degrees",
        "dehydrate",
        "deity",
        "dejected",
        "delayed",
        "demonstrate",
        "dented",
        "deodorant",
        "depth",
        "desk",
        "devoid",
        "dewdrop",
        "dexterity",
        "dialect",
        "dice",
        "diet",
        "different",
        "digit",
        "dilute",
        "dime",
        "dinner",
        "diode",
        "diplomat",
        "directed",
        "distance",
        "ditch",
        "divers",
        "dizzy",
        "doctor",
        "dodge",
        "does",
        "dogs",
        "doing",
        "dolphin",
        "domestic",
        "donuts",
        "doorway",
        "dormant",
        "dosage",
        "dotted",
        "double",
        "dove",
        "down",
        "dozen",
        "dreams",
        "drinks",
        "drowning",
        "drunk",
        "drying",
        "dual",
        "dubbed",
        "duckling",
        "dude",
        "duets",
        "duke",
        "dullness",
        "dummy",
        "dunes",
        "duplex",
        "duration",
        "dusted",
        "duties",
        "dwarf",
        "dwelt",
        "dwindling",
        "dying",
        "dynamite",
        "dyslexic",
        "each",
        "eagle",
        "earth",
        "easy",
        "eating",
        "eavesdrop",
        "eccentric",
        "echo",
        "eclipse",
        "economics",
        "ecstatic",
        "eden",
        "edgy",
        "edited",
        "educated",
        "eels",
        "efficient",
        "eggs",
        "egotistic",
        "eight",
        "either",
        "eject",
        "elapse",
        "elbow",
        "eldest",
        "eleven",
        "elite",
        "elope",
        "else",
        "eluded",
        "emails",
        "ember",
        "emerge",
        "emit",
        "emotion",
        "empty",
        "emulate",
        "energy",
        "enforce",
        "enhanced",
        "enigma",
        "enjoy",
        "enlist",
        "enmity",
        "enough",
        "enraged",
        "ensign",
        "entrance",
        "envy",
        "epoxy",
        "equip",
        "erase",
        "erected",
        "erosion",
        "error",
        "eskimos",
        "espionage",
        "essential",
        "estate",
        "etched",
        "eternal",
        "ethics",
        "etiquette",
        "evaluate",
        "evenings",
        "evicted",
        "evolved",
        "examine",
        "excess",
        "exhale",
        "exit",
        "exotic",
        "exquisite",
        "extra",
        "exult",
        "fabrics",
        "factual",
        "fading",
        "fainted",
        "faked",
        "fall",
        "family",
        "fancy",
        "farming",
        "fatal",
        "faulty",
        "fawns",
        "faxed",
        "fazed",
        "feast",
        "february",
        "federal",
        "feel",
        "feline",
        "females",
        "fences",
        "ferry",
        "festival",
        "fetches",
        "fever",
        "fewest",
        "fiat",
        "fibula",
        "fictional",
        "fidget",
        "fierce",
        "fifteen",
        "fight",
        "films",
        "firm",
        "fishing",
        "fitting",
        "five",
        "fixate",
        "fizzle",
        "fleet",
        "flippant",
        "flying",
        "foamy",
        "focus",
        "foes",
        "foggy",
        "foiled",
        "folding",
        "fonts",
        "foolish",
        "fossil",
        "fountain",
        "fowls",
        "foxes",
        "foyer",
        "framed",
        "friendly",
        "frown",
        "fruit",
        "frying",
        "fudge",
        "fuel",
        "fugitive",
        "fully",
        "fuming",
        "fungal",
        "furnished",
        "fuselage",
        "future",
        "fuzzy",
        "gables",
        "gadget",
        "gags",
        "gained",
        "galaxy",
        "gambit",
        "gang",
        "gasp",
        "gather",
        "gauze",
        "gave",
        "gawk",
        "gaze",
        "gearbox",
        "gecko",
        "geek",
        "gels",
        "gemstone",
        "general",
        "geometry",
        "germs",
        "gesture",
        "getting",
        "geyser",
        "ghetto",
        "ghost",
        "giant",
        "giddy",
        "gifts",
        "gigantic",
        "gills",
        "gimmick",
        "ginger",
        "girth",
        "giving",
        "glass",
        "gleeful",
        "glide",
        "gnaw",
        "gnome",
        "goat",
        "goblet",
        "godfather",
        "goes",
        "goggles",
        "going",
        "goldfish",
        "gone",
        "goodbye",
        "gopher",
        "gorilla",
        "gossip",
        "gotten",
        "gourmet",
        "governing",
        "gown",
        "greater",
        "grunt",
        "guarded",
        "guest",
        "guide",
        "gulp",
        "gumball",
        "guru",
        "gusts",
        "gutter",
        "guys",
        "gymnast",
        "gypsy",
        "gyrate",
        "habitat",
        "hacksaw",
        "haggled",
        "hairy",
        "hamburger",
        "happens",
        "hashing",
        "hatchet",
        "haunted",
        "having",
        "hawk",
        "haystack",
        "hazard",
        "hectare",
        "hedgehog",
        "heels",
        "hefty",
        "height",
        "hemlock",
        "hence",
        "heron",
        "hesitate",
        "hexagon",
        "hickory",
        "hiding",
        "highway",
        "hijack",
        "hiker",
        "hills",
        "himself",
        "hinder",
        "hippo",
        "hire",
        "history",
        "hitched",
        "hive",
        "hoax",
        "hobby",
        "hockey",
        "hoisting",
        "hold",
        "honked",
        "hookup",
        "hope",
        "hornet",
        "hospital",
        "hotel",
        "hounded",
        "hover",
        "howls",
        "hubcaps",
        "huddle",
        "huge",
        "hull",
        "humid",
        "hunter",
        "hurried",
        "husband",
        "huts",
        "hybrid",
        "hydrogen",
        "hyper",
        "iceberg",
        "icing",
        "icon",
        "identity",
        "idiom",
        "idled",
        "idols",
        "igloo",
        "ignore",
        "iguana",
        "illness",
        "imagine",
        "imbalance",
        "imitate",
        "impel",
        "inactive",
        "inbound",
        "incur",
        "industrial",
        "inexact",
        "inflamed",
        "ingested",
        "initiate",
        "injury",
        "inkling",
        "inline",
        "inmate",
        "innocent",
        "inorganic",
        "input",
        "inquest",
        "inroads",
        "insult",
        "intended",
        "inundate",
        "invoke",
        "inwardly",
        "ionic",
        "irate",
        "iris",
        "irony",
        "irritate",
        "island",
        "isolated",
        "issued",
        "italics",
        "itches",
        "items",
        "itinerary",
        "itself",
        "ivory",
        "jabbed",
        "jackets",
        "jaded",
        "jagged",
        "jailed",
        "jamming",
        "january",
        "jargon",
        "jaunt",
        "javelin",
        "jaws",
        "jazz",
        "jeans",
        "jeers",
        "jellyfish",
        "jeopardy",
        "jerseys",
        "jester",
        "jetting",
        "jewels",
        "jigsaw",
        "jingle",
        "jittery",
        "jive",
        "jobs",
        "jockey",
        "jogger",
        "joining",
        "joking",
        "jolted",
        "jostle",
        "journal",
        "joyous",
        "jubilee",
        "judge",
        "juggled",
        "juicy",
        "jukebox",
        "july",
        "jump",
        "junk",
        "jury",
        "justice",
        "juvenile",
        "kangaroo",
        "karate",
        "keep",
        "kennel",
        "kept",
        "kernels",
        "kettle",
        "keyboard",
        "kickoff",
        "kidneys",
        "king",
        "kiosk",
        "kisses",
        "kitchens",
        "kiwi",
        "knapsack",
        "knee",
        "knife",
        "knowledge",
        "knuckle",
        "koala",
        "laboratory",
        "ladder",
        "lagoon",
        "lair",
        "lakes",
        "lamb",
        "language",
        "laptop",
        "large",
        "last",
        "later",
        "launching",
        "lava",
        "lawsuit",
        "layout",
        "lazy",
        "lectures",
        "ledge",
        "leech",
        "left",
        "legion",
        "leisure",
        "lemon",
        "lending",
        "leopard",
        "lesson",
        "lettuce",
        "lexicon",
        "liar",
        "library",
        "licks",
        "lids",
        "lied",
        "lifestyle",
        "light",
        "likewise",
        "lilac",
        "limits",
        "linen",
        "lion",
        "lipstick",
        "liquid",
        "listen",
        "lively",
        "loaded",
        "lobster",
        "locker",
        "lodge",
        "lofty",
        "logic",
        "loincloth",
        "long",
        "looking",
        "lopped",
        "lordship",
        "losing",
        "lottery",
        "loudly",
        "love",
        "lower",
        "loyal",
        "lucky",
        "luggage",
        "lukewarm",
        "lullaby",
        "lumber",
        "lunar",
        "lurk",
        "lush",
        "luxury",
        "lymph",
        "lynx",
        "lyrics",
        "macro",
        "madness",
        "magically",
        "mailed",
        "major",
        "makeup",
        "malady",
        "mammal",
        "maps",
        "masterful",
        "match",
        "maul",
        "maverick",
        "maximum",
        "mayor",
        "maze",
        "meant",
        "mechanic",
        "medicate",
        "meeting",
        "megabyte",
        "melting",
        "memoir",
        "menu",
        "merger",
        "mesh",
        "metro",
        "mews",
        "mice",
        "midst",
        "mighty",
        "mime",
        "mirror",
        "misery",
        "mittens",
        "mixture",
        "moat",
        "mobile",
        "mocked",
        "mohawk",
        "moisture",
        "molten",
        "moment",
        "money",
        "moon",
        "mops",
        "morsel",
        "mostly",
        "motherly",
        "mouth",
        "movement",
        "mowing",
        "much",
        "muddy",
        "muffin",
        "mugged",
        "mullet",
        "mumble",
        "mundane",
        "muppet",
        "mural",
        "musical",
        "muzzle",
        "myriad",
        "mystery",
        "myth",
        "nabbing",
        "nagged",
        "nail",
        "names",
        "nanny",
        "napkin",
        "narrate",
        "nasty",
        "natural",
        "nautical",
        "navy",
        "nearby",
        "necklace",
        "needed",
        "negative",
        "neither",
        "neon",
        "nephew",
        "nerves",
        "nestle",
        "network",
        "neutral",
        "never",
        "newt",
        "nexus",
        "nibs",
        "niche",
        "niece",
        "nifty",
        "nightly",
        "nimbly",
        "nineteen",
        "nirvana",
        "nitrogen",
        "nobody",
        "nocturnal",
        "nodes",
        "noises",
        "nomad",
        "noodles",
        "northern",
        "nostril",
        "noted",
        "nouns",
        "novelty",
        "nowhere",
        "nozzle",
        "nuance",
        "nucleus",
        "nudged",
        "nugget",
        "nuisance",
        "null",
        "number",
        "nuns",
        "nurse",
        "nutshell",
        "nylon",
        "oaks",
        "oars",
        "oasis",
        "oatmeal",
        "obedient",
        "object",
        "obliged",
        "obnoxious",
        "observant",
        "obtains",
        "obvious",
        "occur",
        "ocean",
        "october",
        "odds",
        "odometer",
        "offend",
        "often",
        "oilfield",
        "ointment",
        "okay",
        "older",
        "olive",
        "olympics",
        "omega",
        "omission",
        "omnibus",
        "onboard",
        "oncoming",
        "oneself",
        "ongoing",
        "onion",
        "online",
        "onslaught",
        "onto",
        "onward",
        "oozed",
        "opacity",
        "opened",
        "opposite",
        "optical",
        "opus",
        "orange",
        "orbit",
        "orchid",
        "orders",
        "organs",
        "origin",
        "ornament",
        "orphans",
        "oscar",
        "ostrich",
        "otherwise",
        "otter",
        "ouch",
        "ought",
        "ounce",
        "ourselves",
        "oust",
        "outbreak",
        "oval",
        "oven",
        "owed",
        "owls",
        "owner",
        "oxidant",
        "oxygen",
        "oyster",
        "ozone",
        "pact",
        "paddles",
        "pager",
        "pairing",
        "palace",
        "pamphlet",
        "pancakes",
        "paper",
        "paradise",
        "pastry",
        "patio",
        "pause",
        "pavements",
        "pawnshop",
        "payment",
        "peaches",
        "pebbles",
        "peculiar",
        "pedantic",
        "peeled",
        "pegs",
        "pelican",
        "pencil",
        "people",
        "pepper",
        "perfect",
        "pests",
        "petals",
        "phase",
        "pheasants",
        "phone",
        "phrases",
        "physics",
        "piano",
        "picked",
        "pierce",
        "pigment",
        "piloted",
        "pimple",
        "pinched",
        "pioneer",
        "pipeline",
        "pirate",
        "pistons",
        "pitched",
        "pivot",
        "pixels",
        "pizza",
        "playful",
        "pledge",
        "pliers",
        "plotting",
        "plus",
        "plywood",
        "poaching",
        "pockets",
        "podcast",
        "poetry",
        "point",
        "poker",
        "polar",
        "ponies",
        "pool",
        "popular",
        "portents",
        "possible",
        "potato",
        "pouch",
        "poverty",
        "powder",
        "pram",
        "present",
        "pride",
        "problems",
        "pruned",
        "prying",
        "psychic",
        "public",
        "puck",
        "puddle",
        "puffin",
        "pulp",
        "pumpkins",
        "punch",
        "puppy",
        "purged",
        "push",
        "putty",
        "puzzled",
        "pylons",
        "pyramid",
        "python",
        "queen",
        "quick",
        "quote",
        "rabbits",
        "racetrack",
        "radar",
        "rafts",
        "rage",
        "railway",
        "raking",
        "rally",
        "ramped",
        "randomly",
        "rapid",
        "rarest",
        "rash",
        "rated",
        "ravine",
        "rays",
        "razor",
        "react",
        "rebel",
        "recipe",
        "reduce",
        "reef",
        "refer",
        "regular",
        "reheat",
        "reinvest",
        "rejoices",
        "rekindle",
        "relic",
        "remedy",
        "renting",
        "reorder",
        "repent",
        "request",
        "reruns",
        "rest",
        "return",
        "reunion",
        "revamp",
        "rewind",
        "rhino",
        "rhythm",
        "ribbon",
        "richly",
        "ridges",
        "rift",
        "rigid",
        "rims",
        "ringing",
        "riots",
        "ripped",
        "rising",
        "ritual",
        "river",
        "roared",
        "robot",
        "rockets",
        "rodent",
        "rogue",
        "roles",
        "romance",
        "roomy",
        "roped",
        "roster",
        "rotate",
        "rounded",
        "rover",
        "rowboat",
        "royal",
        "ruby",
        "rudely",
        "ruffled",
        "rugged",
        "ruined",
        "ruling",
        "rumble",
        "runway",
        "rural",
        "rustled",
        "ruthless",
        "sabotage",
        "sack",
        "sadness",
        "safety",
        "saga",
        "sailor",
        "sake",
        "salads",
        "sample",
        "sanity",
        "sapling",
        "sarcasm",
        "sash",
        "satin",
        "saucepan",
        "saved",
        "sawmill",
        "saxophone",
        "sayings",
        "scamper",
        "scenic",
        "school",
        "science",
        "scoop",
        "scrub",
        "scuba",
        "seasons",
        "second",
        "sedan",
        "seeded",
        "segments",
        "seismic",
        "selfish",
        "semifinal",
        "sensible",
        "september",
        "sequence",
        "serving",
        "session",
        "setup",
        "seventh",
        "sewage",
        "shackles",
        "shelter",
        "shipped",
        "shocking",
        "shrugged",
        "shuffled",
        "shyness",
        "siblings",
        "sickness",
        "sidekick",
        "sieve",
        "sifting",
        "sighting",
        "silk",
        "simplest",
        "sincerely",
        "sipped",
        "siren",
        "situated",
        "sixteen",
        "sizes",
        "skater",
        "skew",
        "skirting",
        "skulls",
        "skydive",
        "slackens",
        "sleepless",
        "slid",
        "slower",
        "slug",
        "smash",
        "smelting",
        "smidgen",
        "smog",
        "smuggled",
        "snake",
        "sneeze",
        "sniff",
        "snout",
        "snug",
        "soapy",
        "sober",
        "soccer",
        "soda",
        "software",
        "soggy",
        "soil",
        "solved",
        "somewhere",
        "sonic",
        "soothe",
        "soprano",
        "sorry",
        "southern",
        "sovereign",
        "sowed",
        "soya",
        "space",
        "speedy",
        "sphere",
        "spiders",
        "splendid",
        "spout",
        "sprig",
        "spud",
        "spying",
        "square",
        "stacking",
        "stellar",
        "stick",
        "stockpile",
        "strained",
        "stunning",
        "stylishly",
        "subtly",
        "succeed",
        "suddenly",
        "suede",
        "suffice",
        "sugar",
        "suitcase",
        "sulking",
        "summon",
        "sunken",
        "superior",
        "surfer",
        "sushi",
        "suture",
        "swagger",
        "swept",
        "swiftly",
        "sword",
        "swung",
        "syllabus",
        "symptoms",
        "syndrome",
        "syringe",
        "system",
        "taboo",
        "tacit",
        "tadpoles",
        "tagged",
        "tail",
        "taken",
        "talent",
        "tamper",
        "tanks",
        "tapestry",
        "tarnished",
        "tasked",
        "tattoo",
        "taunts",
        "tavern",
        "tawny",
        "taxi",
        "teardrop",
        "technical",
        "tedious",
        "teeming",
        "tell",
        "template",
        "tender",
        "tepid",
        "tequila",
        "terminal",
        "testing",
        "tether",
        "textbook",
        "thaw",
        "theatrics",
        "thirsty",
        "thorn",
        "threaten",
        "thumbs",
        "thwart",
        "ticket",
        "tidy",
        "tiers",
        "tiger",
        "tilt",
        "timber",
        "tinted",
        "tipsy",
        "tirade",
        "tissue",
        "titans",
        "toaster",
        "tobacco",
        "today",
        "toenail",
        "toffee",
        "together",
        "toilet",
        "token",
        "tolerant",
        "tomorrow",
        "tonic",
        "toolbox",
        "topic",
        "torch",
        "tossed",
        "total",
        "touchy",
        "towel",
        "toxic",
        "toyed",
        "trash",
        "trendy",
        "tribal",
        "trolling",
        "truth",
        "trying",
        "tsunami",
        "tubes",
        "tucks",
        "tudor",
        "tuesday",
        "tufts",
        "tugs",
        "tuition",
        "tulips",
        "tumbling",
        "tunnel",
        "turnip",
        "tusks",
        "tutor",
        "tuxedo",
        "twang",
        "tweezers",
        "twice",
        "twofold",
        "tycoon",
        "typist",
        "tyrant",
        "ugly",
        "ulcers",
        "ultimate",
        "umbrella",
        "umpire",
        "unafraid",
        "unbending",
        "uncle",
        "under",
        "uneven",
        "unfit",
        "ungainly",
        "unhappy",
        "union",
        "unjustly",
        "unknown",
        "unlikely",
        "unmask",
        "unnoticed",
        "unopened",
        "unplugs",
        "unquoted",
        "unrest",
        "unsafe",
        "until",
        "unusual",
        "unveil",
        "unwind",
        "unzip",
        "upbeat",
        "upcoming",
        "update",
        "upgrade",
        "uphill",
        "upkeep",
        "upload",
        "upon",
        "upper",
        "upright",
        "upstairs",
        "uptight",
        "upwards",
        "urban",
        "urchins",
        "urgent",
        "usage",
        "useful",
        "usher",
        "using",
        "usual",
        "utensils",
        "utility",
        "utmost",
        "utopia",
        "uttered",
        "vacation",
        "vague",
        "vain",
        "value",
        "vampire",
        "vane",
        "vapidly",
        "vary",
        "vastness",
        "vats",
        "vaults",
        "vector",
        "veered",
        "vegan",
        "vehicle",
        "vein",
        "velvet",
        "venomous",
        "verification",
        "vessel",
        "veteran",
        "vexed",
        "vials",
        "vibrate",
        "victim",
        "video",
        "viewpoint",
        "vigilant",
        "viking",
        "village",
        "vinegar",
        "violin",
        "vipers",
        "virtual",
        "visited",
        "vitals",
        "vivid",
        "vixen",
        "vocal",
        "vogue",
        "voice",
        "volcano",
        "vortex",
        "voted",
        "voucher",
        "vowels",
        "voyage",
        "vulture",
        "wade",
        "waffle",
        "wagtail",
        "waist",
        "waking",
        "wallets",
        "wanted",
        "warped",
        "washing",
        "water",
        "waveform",
        "waxing",
        "wayside",
        "weavers",
        "website",
        "wedge",
        "weekday",
        "weird",
        "welders",
        "went",
        "wept",
        "were",
        "western",
        "wetsuit",
        "whale",
        "when",
        "whipped",
        "whole",
        "wickets",
        "width",
        "wield",
        "wife",
        "wiggle",
        "wildly",
        "winter",
        "wipeout",
        "wiring",
        "wise",
        "withdrawn",
        "wives",
        "wizard",
        "wobbly",
        "woes",
        "woken",
        "wolf",
        "womanly",
        "wonders",
        "woozy",
        "worry",
        "wounded",
        "woven",
        "wrap",
        "wrist",
        "wrong",
        "yacht",
        "yahoo",
        "yanks",
        "yard",
        "yawning",
        "yearbook",
        "yellow",
        "yesterday",
        "yeti",
        "yields",
        "yodel",
        "yoga",
        "younger",
        "yoyo",
        "zapped",
        "zeal",
        "zebra",
        "zero",
        "zesty",
        "zigzags",
        "zinger",
        "zippers",
        "zodiac",
        "zombie",
        "zones",
        "zoom",
    ]

In [ ]:
'''
Zero to Monero: Second Edition; Section 4.1
'''


'''
To calculate the private spend key this code performs the following:

  1. Extracts three word blocks
  2. Converts the three word blocks to a number
  3. Converts the number to a little-endian encoded byte array
  4. Converts the byte array to hex
  5. Joins the blocks together into one hex string
'''

# define a function to convert the 24 word mnemonic to private spend key
def mnemonic_to_private_spend_key(mnemonic: list, word_list: list) -> str:
    private_spend_key = ""
    
    # the word list contains 1626 words
    len_word_list = len(word_list)

    # iterate over the mnemonic three words at a time (8 iterations * 3 words = 24 words)
    for i in range(len(mnemonic) // 3):
        
        # extract a group of three words from the mnemonic
        word1, word2, word3 = mnemonic[3 * i : 3 * i + 3]

        # convert the three words to three numbers       
        number1 = word_list.index(word1)
        number2 = word_list.index(word2)
        number3 = word_list.index(word3)
        print_str = f"Inside function: iteration {i} word list {word1, word2, word3}, corresponding numbers {number1, number2, number3}"

        # use the three numbers to create a single number that fills four bytes of data
        transformed_number1 = number1
        transformed_number2 = len_word_list * ((number2 - number1) % len_word_list)
        transformed_number3 = len_word_list * len_word_list * ((number3 - number2) % len_word_list)
        transformed_numbers_added = transformed_number1 + transformed_number2 + transformed_number3
        print_str += f", transformed numbers {transformed_number1, transformed_number2, transformed_number3}, added {transformed_numbers_added}"

        # convert the four byte number to a little endian byte string
        transformed_numbers_added = int(transformed_numbers_added).to_bytes(4, byteorder="little")
        print_str += f", little-endian bytes {transformed_numbers_added}"

        # convert the little endian four byte string to an eight character hex string
        transformed_numbers_added = transformed_numbers_added.hex()
        print_str += f", hex version {transformed_numbers_added}"
        print(print_str)

        # build up the private spend key 8 hex characters at a time
        private_spend_key += transformed_numbers_added
    
    return private_spend_key


# run the function and store the private spend key
private_spend_key = mnemonic_to_private_spend_key(alice_mnemonic, word_list)


print(f"Alice's private spend key is: {private_spend_key}")

## 3. Calculating the private view key

The Monero private view key is derived from the private spend key by hashing the spend key. Private keys are large numbers constrained to a specific range of values. The number calculated by hashing the private spend key may not be within the allowable range so another function is used on the hashed value to put it within the valid range of values if it isn't already.

In [ ]:
'''
Zero to Monero: Second Edition; Section 4.1
'''


'''
To calculate the private view key this code performs the following:

  1. Uses the standard Monero hash (Keccak) to produce a hash of the private spend key
  2. Ensures the hash is within the valid range of private view key numbers
  3. Converts the number to a hex string
'''

# import required libraries (libraries specific to each section will be imported/re-imported for clarity)
import nacl.bindings

from Cryptodome.Hash import keccak
from binascii import hexlify, unhexlify


# define a function to return a hashed byte string
def keccak_256(data):
    return keccak.new(digest_bits=256).update(data).digest()

# define a function to convert an integer into a valid ed25519 scalar
def pad_and_reduce(scalar: bytes) -> bytes:
    point_padded_to_64_bytes = scalar + (64 - len(scalar)) * b"\0"
    return nacl.bindings.crypto_core_ed25519_scalar_reduce(point_padded_to_64_bytes)


# convert private spend key hex string data to byte string (note: unhexlify() = bytes.fromhex())
private_view_key = unhexlify(private_spend_key)

# keccak 256 hash the byte private spend key (byte string))
private_view_key = keccak_256(private_view_key)

# ensure hash is a valid ed25519 scalar (byte string)
private_view_key = pad_and_reduce(private_view_key)

# convert byte string to hex string (note: hexlify().decode() = bytes.hex())
private_view_key = hexlify(private_view_key).decode()


print(f"Alice's private view key is: {private_view_key}")

## 4. Calculate the public spend key

In elliptic curve cryptography the private key is an integer and the public key is a point on a curve (literally a point with (x,y) coordinates). The valid (x,y) coordinates exist on a specific curve used by Monero knows as Ed25519. Points on the curve can be added to themself or other points on the curve any number of times via special math operations to produce other points on the curve. There is a special point on the curve known as the generator point "G". The generator point is publicly known and the coordinates are accessible by anyone. Adding "G" to itself twice can be written as 2G. Adding "G to itself 100 times can be written as 100G. Calculating 100G is very easy, but calculating 100 if given 100G is not easy. 

There are nearly as many points on the curve as there are atoms in the observable universe, so in reality it isn't 100G but something closer to 22145104581983974271410479856667069913544659902559263543006111805898435315717G. The number multiplying "G" is the private key and the resulting curve point (remember "G" is a point on the curve with (x,y) coordinates) is the public key. Often the private key is written as a lower case letter k and the public key is written as an upper case letter K. That is what is meant by kG = K.  

Alice is easily able to multiply "G" by her private key (k) to receive her public key (k). However, Bob is not able to recover the private key (k) from the public key (K). This is true because kG = K, which represents a number (k) multiplying a curve point (K), does not behave like simply multiplying two numbers. It is not possible to do (kG)/G = K/G; k = K/G. This operation not being possible makes public/private keys work.  

In [ ]:
'''
Zero to Monero: Second Edition; Section 4.1
'''


'''
To calculate the public spend key this code performs the following:

  1. Multiplies the private spend key by the generator point "G"
  2. Converts the resulting curve point to a hex string
'''

# import required libraries (libraries specific to each section will be imported/re-imported for clarity)
import nacl.bindings
from binascii import hexlify, unhexlify


# create function to multiply the ed25519 generator point (G) by a scalar
scalar_mult_G = nacl.bindings.crypto_scalarmult_ed25519_base_noclamp


# convert private spend key hex string to byte string
public_spend_key_unhex = unhexlify(private_spend_key)

# multiply the ed25519 base point G by the private spend key (creates a curve point compressed to a single byte string)
public_spend_key = scalar_mult_G(public_spend_key_unhex)

# convert the curve point byte string to hex string
public_spend_key = hexlify(public_spend_key).decode()


print(f"Alice's public spend key is: {public_spend_key}")

## 5. Calculate the public view key

The public view key is calculated using the same operations as the public spend key, but using the private view key instead.

In [ ]:
'''
Zero to Monero: Second Edition; Section 4.1
'''


'''
To calculate the public view key this code performs the following:

  1. Multiply the private view key by the generator point "G"
  2. Convert the resulting curve point to a hex string
'''


# import required libraries (libraries specific to each section will be imported/re-imported for clarity)
import nacl.bindings
from binascii import hexlify, unhexlify


# create function to multiply the ed25519 generator point (G) by a scalar
scalarmult_G = nacl.bindings.crypto_scalarmult_ed25519_base_noclamp


# convert private view key hex string to byte string
public_view_key_unhex = unhexlify(private_view_key)

# multiply the ed25519 base point G by the private view key (creates a curve point compressed to a single byte string)
public_view_key = scalarmult_G(public_view_key_unhex)

# convert the curve point byte string to hex string
public_view_key = hexlify(public_view_key).decode()


print(f"Alice's public view key is: {public_view_key}")

## 6. Convert the public spend/view keys into a Monero address

The public keys are used by senders to create a transaction for recipients. A Monero address contains these two pieces of information as well as an indicator to show if the address is meant for the main Monero network or for staging/test environments. The address also contains a checksum. To summarise, a Monero address includes four pieces of information:

* net byte showing which network the address represents (mainnet, staging, dev)
* public spend key
* public view key
* checksum from hashing the above

The above information is concatenated together and then converted to base58 encoding. Base58 encoding ensures that similar looking characters are not used (ex: uppercase "i", lowercase "l").

In [ ]:
'''
Zero to Monero: Second Edition; Section 4.1
'''


'''
To calculate the public view key this code performs the following:

  1. Concatenates net byte + public spend key + public view key
  2. Calculates checksum from above
  3. Concatenates step 1 and checksum
  4. Loops through 8 byte blocks of data (blocks here are completely unrelated to blockchain blocks)
  5. Converts 8 bytes to integer
  6. Converts integer to base58 encoding
  7. Concatenates all base58 encoded values
'''

from Cryptodome.Hash import keccak
from binascii import unhexlify


# define a function to return a hashed byte string
def keccak_256(data) -> bytes:
    return keccak.new(digest_bits=256).update(data).hexdigest()


# define a function to convert hex to base58 encoding
def hex_to_base58(hex_str: str) -> str:
    # Define the Base58 alphabet (58 characters)
    alphabet = "123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz"

    # Define the block size, which is the number of bytes that will be encoded together in each block. In this case, the block size is 8 bytes.
    block_size = 8

    # Convert the hexadecimal string to a byte array using the bytes.fromhex() method, which returns a bytes object.
    byte_arr = bytes.fromhex(hex_str)
    print(f"Netbyte + keys + checksum as bytes {byte_arr}")

    # Determine the number of blocks by dividing the length of the byte array by the block size and rounding up to the nearest integer using the // operator.
    block_count = (len(byte_arr) + block_size - 1) // block_size

    # For each block, encode it using the Base58 alphabet. 
    # To do this, we first convert the block to a number using the int.from_bytes() method with the byteorder parameter set to "big" to ensure the most significant byte is first. 
    # We then perform base conversion on this number to obtain the Base58 encoded string.
    encoded_blocks = []
    for i in range(block_count):
        block_start = i * block_size
        block_end = min(block_start + block_size, len(byte_arr))
        block = byte_arr[block_start:block_end]
        num = int.from_bytes(block, byteorder="big")
        num_for_print = num
        encoded = ""
        while num > 0:
            num, remainder = divmod(num, len(alphabet))
            encoded = alphabet[remainder] + encoded
        
        print(f"Inside function - block of bytes: {block} corresponding number: {num_for_print} base58 encoded number: {encoded}")
        # Append the encoded block to a list of encoded blocks.
        encoded_blocks.append(encoded)

    # Join the encoded blocks using an empty string as the separator to obtain the final Base58 encoded string, and return it.
    return "".join(encoded_blocks)


# indicator for main, dev, staging (main = 18, dev =, staging = 35, testnet = 53)
netbyte = 18
address = "{:x}{:s}{:s}".format(netbyte, public_spend_key, public_view_key)

# calculate keccak 256 hash as a hex string
checksum = keccak_256(unhexlify(address))
checksum = checksum[0:8]

# convert the hex string to a base58 string
address = hex_to_base58(address + checksum)


print(f"Alice's Monero address is: {address}")

## 7. Convert a Monero address into public spend/view keys

The above process can be inverted such that the four pieces of information (net byte, public spend/view keys, checksum) can be recovered from the Monero address. The Monero address was created by working on 8 byte blocks of data and converting each 8 byte block to base58 encoding. An 8 byte block of data contains 64 bits of data. Bits, hexadecimal, and bytes are all based on the 2 to some power, but base58 is not based on 2 to some power.  

One hexadecimal character represents one of 16 values. That number is a power of 2, specifically $2^4$, which means one hexadecimal character can hold 4 bits of data. Two hexadecimal characters can hold 8 bits (1 byte) of data. For further clarity 16 hexadecimal characters can hold 64 bits (8 bytes) of data. To summarise:

* one hex character = 4 bits
* two hex characters = 8 bits = 1 byte
* fourteen hex characters = 56 bits = 7 bytes
* fifteen hex characters = 60 bits = 7.5 bytes
* sixteen hex characters = 64 bits = 8 bytes

One base 58 character represents one of 58 characters which is more than what one hex character can represent but less than what two hex characters can represent. One hex character can represent one of 16 unique values while two hex characters can represent one of 64 unique characters. 

Originally 8 bytes blocks of data (16 hex characters) were operated on to create the base58 encoding. A block of 8 bytes has 64 bits which can hold a certain number of unique values. A block of 10 base58 characters can hold less unique values than 8 bytes while 11 base58 characters can hold more unique values than 8 bytes.

* 8 byte block = 64 bits: $2^{64} = 18446744073709551616$
* 10 base58 characters: $58^{10} = 430804206899405824$
* 11 base58 characters: $58^{11} = 24986644000165537792$

Because 10 base58 characters < 8 bytes < 11 base58 characters, the process to recover the keys from the Monero address will use blocks of 11 base58 characters.


In [ ]:
'''
Zero to Monero: Second Edition; Section 4.1
'''


'''
To calculate the public view key this code performs the following:

  1. Defines the base58 "alphabet"
  2. Iterates over the Monero address
  3. Converts 11 base58 characters at a time to an integer
  4. Converts the integer to big-endian byte array
  5. Converts the byte array to hex
  6. Concatenates all of the create hex strings
  7. Extracts the net byte + public spend key + public view key + checksum
'''

import math

def base58_to_hex(encoded_str):
    # Define the base58 alphabet and initialize some variables
    alphabet = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
    base58_alphabet_size = len(alphabet)
    num = 0
    count = 0
    result = ''

    # Iterate over each character in the encoded string
    for i, char in enumerate(encoded_str):
        # Find the index of the current character in the alphabet
        base58_index = alphabet.index(char)
 
        # Multiply the current result by the base and add the current index
        num = num * base58_alphabet_size + base58_index
        count += 1

        # If we've processed 11 characters or reached the end of the string,
        # convert the result to a big-endian byte array and then to hex
        if count == 11 or i == len(encoded_str) - 1:
            
            # Convert the byte array to a hex string and add it to the result
            bits_in_number = num.bit_length()
            bytes_in_number = math.ceil(bits_in_number / 8.0)
            number_as_hex = num.to_bytes(bytes_in_number, byteorder="big").hex()
            print(f"Inside function - base58 block to integer: {num} bytes in number: {bytes_in_number} number as hex: {number_as_hex}")
            result += number_as_hex
            
            # Reset the variables for the next block
            num = 0
            count = 0

    return result

address_hex = base58_to_hex("49JpAeZ6WGHSXaRD5NVcNQHtqVeVinUTLRzjzwpa4NCLAKbjKY3M1NHAE1AAfUckpFJypVabwrKWnEeEXhAVLmk3CfRTwPb")

netbyte_recovered = address_hex[:2]
public_spend_key_recovered = address_hex[2:66]
public_view_key_recovered = address_hex[66:130]
checksum_recovered = address_hex[130:]

print(f"Alice's public spend key (recovered from address) is: {public_spend_key_recovered}")
print(f"Alice's public view key (recovered from address) is: {public_view_key_recovered}")

## 8. Show public key curve points (optional)

This section is entirely optional. It was only added to clarify that a public key (curve point) literally represents (x,y) coordinates in the 2-d plane. Alice's public view/spend keys are literally x and y coordinates in the 2-d plane on the Ed25519 curve. The code displayed is quite complicated and is recreated from the book [Zero to Monero: Second Edition; Chapter 2](https://www.getmonero.org/library/Zero-to-Monero-2-0-0.pdf).

A Monero public key directly encodes the y coordinate. For each y-coordinate there are two possible x-coordinates. The public key includes an extra 0/1 bit to indicate which of the two x-coordinates was used.

In [ ]:
'''
Zero to Monero: Second Edition; Section 2.4.2
'''


'''
To calculate curve points this code performs the following:

  1. Extracts the first bit from the public key
  2. Uses the remaining bits at the y-value
  3. Calculates an x-value
  4. Checks if the x-value is positive or negative
  5. Compares step 4 to the bit from step 2
  6. Modifies x as necessary to be consistent with the first indicator bit
  7. Retuns (x,y) values
'''

# define q = prime_field
prime_field = 2**255 - 19


# define "d"
d = -(121665 * pow(121666, -1, prime_field)) % prime_field


# define a function to recreate G
def recreate_G():
    # G is the compressed curve point but since compressed bit identifier is zero, just use y = (4/5) mod q
    G = (4 * pow(5, -1, prime_field)) % prime_field
    return G.to_bytes(32, byteorder="little").hex()


# define a function to recover (x,y) coordinates from a public key (which is a compressed curve point)
def public_key_to_xy(public_key: str):
    # Convert the hex string to bytes
    y_bytes = bytes.fromhex(public_key)

    # Use the hex str as the y coordinate in the ed25519 curve
    y = int.from_bytes(y_bytes, byteorder="little")

    # copy MSB to parity bit "b"
    parity_bit = (y >> 255) & 0x01

    # Change the most significant bit to 0
    y &= ~(1 << 255)

    # Define the ed25519 curve parameters
    u = (pow(y, 2, prime_field) - 1) % prime_field
    v = ((d * pow(y, 2, prime_field)) % prime_field + 1) % prime_field
    z = (
            ((u * pow(v, 3, prime_field)) % prime_field) * 
            pow( 
                ((u * pow(v, 7, prime_field)) % prime_field), 
                (((prime_field - 5) * pow(8, -1, prime_field)) % prime_field),
                prime_field
            )
    ) % prime_field

    v_z2 = (v * pow(z, 2, prime_field)) % prime_field
    u_mod_q = u % prime_field
    neg_u_mod_q = (-u) % prime_field
    
    if v_z2 == u_mod_q:
        x = z
    elif v_z2 == neg_u_mod_q:
        x = (z * pow(2, ((prime_field - 1) * pow(4, -1, prime_field)) % prime_field, prime_field)) % prime_field
    else:
        print("Error")

    # check if LSB in x == parity bit
    if (x & 0x01) != parity_bit:
        x = (-x) % prime_field

    return x, y



# G = "5866666666666666666666666666666666666666666666666666666666666666"
# x: 15112221349535400772501151409588531511454012693041857206046113283949847762202
# y: 46316835694926478169428394003475163141307993866256225615783033603165251855960
G = recreate_G()
x, y = public_key_to_xy(G)
print(f"Generator point -- x: {x}; y: {y}")

x, y = public_key_to_xy(public_spend_key)
print(f"Alice's public spend key -- x: {x}; y: {y}")

x, y = public_key_to_xy(public_view_key)
print(f"Alice's public view key -- x: {x}; y: {y}")


## Summary (Conclusion)
**What we started with:**
* Alice's mnemonic (25 words): `joyous weekday gearbox slower dejected nutshell fuzzy empty wetsuit mighty bawled dummy tufts lawsuit moon bluntly whole cage punch else sayings prying hockey imbalance slower`

**What we ended up with:**
* Alice's private spend key: `30f5af5d3b2a206708c56bb7a2238d2006e47b10cc3ec92b240e1982ecbfc805` (an integer encoded in hexadecimal)
* Alice's private view key: `e04769b2067f76a7942c903f6782e75f2d2dd49437a6bb671e9d8ef09d9fc70b` (an integer encoded in hexadecimal)
* Alice's public spend key: `cac88399eed832989df44ae91a0df5650040be8cfbe21b9570466432e6286d37` (a curve point (x,y) compressed to a single integer encoded in hexadecimal)
* Alice's public view key: `b981d77369cbf23725d61cceb6f6686b7e435ca52e2ccf518f560cc71fc54867` (a curve point (x,y) compressed to a single integer encoded in hexadecimal)
* Alice's Monero address: `49JpAeZ6WGHSXaRD5NVcNQHtqVeVinUTLRzjzwpa4NCLAKbjKY3M1NHAE1AAfUckpFJypVabwrKWnEeEXhAVLmk3CfRTwPb` (18 + public spend key + public view key + checksum encoded in base58)